In [1]:
from os import listdir
from os.path import isfile, join
import pandas as pd
import datetime as dt
import pandas_datareader.data as web

In [2]:
path = r'C:\Users\michael\Desktop\trading materials\us market\uptrend_swing\individual_swing_trade'
txt_files = [f for f in listdir(path) if isfile(join(path, f))]
df_main = pd.DataFrame(columns=['symbol','name'])

for i in txt_files:
    df_txt = pd.read_csv(path + r'\\' + i, index_col = False)
    df_main = df_main.append(df_txt, ignore_index = True)

symbol_list = df_main['symbol'].tolist()

In [3]:
def macd_histogram(ticker):
    end = dt.datetime.today()
    start = dt.datetime.today() - dt.timedelta(days=310)
    df = web.DataReader(ticker, 'yahoo', start, end)
    df['EXP1'] = df['Close'].ewm(span=12, adjust=False).mean()
    df['EXP2'] = df['Close'].ewm(span=26, adjust=False).mean()
    df['MACD_LINE'] = df['EXP1'] - df['EXP2']
    df['SIGNAL_LINE'] = df['MACD_LINE'].ewm(span=9, adjust=False).mean()
    df['MACD_HIST'] = df['MACD_LINE'] - df['SIGNAL_LINE']

    if df['MACD_HIST'].iloc[-1] >= df['MACD_HIST'].iloc[-2] and df['MACD_HIST'].iloc[-2] >= df['MACD_HIST'].iloc[-3]:
        return "Yes"
    else:
        return "No"

In [4]:
macd_result_list = []
for i in symbol_list:
    result = macd_histogram(i)
    macd_result_list.append(result)
df_main['macd increasing'] = pd.Series(macd_result_list)

In [5]:
df = pd.read_csv(r'C:\Users\michael\Desktop\trading materials\us market\uptrend_swing\us tickers.csv')
sector_list = []
for i in symbol_list:
    index = df[df['Symbol'] == i].index.values[0]
    sector = df['Sector'].iloc[index]
    sector_list.append(sector)
df_main['sector'] = pd.Series(sector_list)
df_main = df_main.sort_values(by='macd increasing', ascending=False)
df_main.reset_index(drop=True, inplace=True)
display(df_main)

,symbol,name,macd increasing,sector
0,MCY,Mercury General Corporation Common Stock,Yes,Finance
1,NWSA,News Corporation Class A Common Stock,Yes,Consumer Services
2,DAC,Danaos Corporation Common Stock,Yes,Transportation
3,SSL,Sasol Ltd. American Depositary Shares,Yes,Energy
4,GWB,Great Western Bancorp Inc. Common Stock,Yes,Finance
5,SLF,Sun Life Financial Inc. Common Stock,Yes,Finance
6,JPM,JP Morgan Chase & Co. Common Stock,Yes,Finance
7,COF,Capital One Financial Corporation Common Stock,Yes,Finance
8,RPT,RPT Realty Common Stock,Yes,Consumer Services
9,CAKE,Cheesecake Factory Incorporated (The) Common S...,Yes,Consumer Services


In [6]:
day = dt.datetime.today().weekday()
if day == 5:
    file_date = (dt.datetime.today() + dt.timedelta(days=2)).strftime('%Y%m%d')
elif day == 6:
    file_date = (dt.datetime.today() + dt.timedelta(days=1)).strftime('%Y%m%d')
else:
    file_date = dt.datetime.today().strftime('%Y%m%d')
    
save_path = r'C:\Users\michael\Desktop\trading materials\us market\uptrend_swing_buy' + r'\\' + file_date + '.xlsx' 

df_main.to_excel(save_path, index=False)